Task 1: Generate and load the raw dataset
Load the CSV file data_safe_copy.csv into a DataFrame named tickets. Confirm that it has at least 500 rows. Print the first five rows and run info() to confirm the initial structure.

In [1]:
import pandas as pd
tickets = pd.read_csv("data_safe_copy.csv")
assert tickets.shape[0] >= 500
print("there are at least 500 rows\n")
print("first 5 rows: \n",tickets.head(5),'\n')
print(tickets.info())

there are at least 500 rows

first 5 rows: 
   ticket_id         opened_at   category priority resolution_minutes
0   TK-0001  2024-03-01 09:00    billing      low                 45
1   TK-0002  2024-03-01 09:30    account   medium                 30
2   TK-0003  2024-03-01 10:00  technical     high                 75
3   TK-0004  2024-03-01 10:30    billing      low                NaN
4   TK-0005  2024-03-01 11:00    account   medium            unknown 

<class 'pandas.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   ticket_id           500 non-null    str  
 1   opened_at           500 non-null    str  
 2   category            500 non-null    str  
 3   priority            500 non-null    str  
 4   resolution_minutes  450 non-null    str  
dtypes: str(5)
memory usage: 19.7 KB
None


Task 2: Parse dates and clean categories
Convert opened_at to a datetime column. Then normalize category and priority by stripping whitespace and converting to lowercase. Show the unique values for both columns to confirm cleaning.

In [2]:
tickets["opened_at"]=pd.to_datetime(tickets["opened_at"])
for col in ['category','priority']:
            tickets[col]=tickets[col].str.strip().str.lower()
print("unique category: ",tickets["category"].unique())
print("unique priority: ",tickets["priority"].unique())

unique category:  <StringArray>
['billing', 'account', 'technical']
Length: 3, dtype: str
unique priority:  <StringArray>
['low', 'medium', 'high']
Length: 3, dtype: str


Task 3: Convert resolution time safely
Convert resolution_minutes to numeric values. Invalid entries should become missing values. After conversion, compute the count of missing values and confirm it matches the number of intentionally introduced issues.

In [3]:
print(tickets["resolution_minutes"].unique())
intentionally_issues_count = len(
    tickets[
        (tickets["resolution_minutes"].isna()) |
        (tickets["resolution_minutes"] == "unknown")
    ]
)
tickets["resolution_minutes"] = pd.to_numeric(tickets["resolution_minutes"],errors="coerce")
missing_resolution = tickets["resolution_minutes"].isna().sum()
assert intentionally_issues_count == missing_resolution
print("verification true")

<StringArray>
['45', '30', '75', nan, 'unknown', '60', '15', '90', '120', '25']
Length: 10, dtype: str
verification true


Task 4: Filter and validate cleaned data
Create a cleaned DataFrame named tickets_clean by dropping rows with missing resolution_minutes. Then verify:

All remaining resolution_minutes are non-negative
All categories and priorities are normalized
Show the number of records before and after cleaning.

In [4]:
tickets_clean = tickets.dropna(subset=["resolution_minutes"]).copy()

is_non_negative = (tickets_clean["resolution_minutes"] >= 0).all()

print(f"Records before cleaning: {len(tickets)}")
print(f"Records after cleaning:  {len(tickets_clean)}")
print(f"Non-negative check:      {is_non_negative}")

print("\nValidating Normalized Columns:")
print(f"Unique Categories: {tickets_clean['category'].unique()}")
print(f"Unique Priorities: {tickets_clean['priority'].unique()}")

Records before cleaning: 500
Records after cleaning:  400
Non-negative check:      True

Validating Normalized Columns:
Unique Categories: <StringArray>
['billing', 'account', 'technical']
Length: 3, dtype: str
Unique Priorities: <StringArray>
['low', 'medium', 'high']
Length: 3, dtype: str


Task 5: Build a quality summary
Create a summary dictionary with keys total_records, clean_records, missing_resolution, avg_resolution, and max_resolution. Print the summary and verify that clean_records + missing_resolution equals total_records.

In [5]:
total_records = len(tickets)
clean_records = len(tickets_clean)
missing_resolution = tickets["resolution_minutes"].isna().sum()
avg_resolution = tickets_clean["resolution_minutes"].mean()
max_resolution = tickets_clean["resolution_minutes"].max()

summary = {
    "total_records": total_records,
    "clean_records": clean_records,
    "missing_resolution": missing_resolution,
    "avg_resolution": avg_resolution,
    "max_resolution": max_resolution
}
is_consistent = summary["clean_records"] + summary["missing_resolution"] == summary["total_records"]

print("Summary:", summary)
print("Consistency check:", is_consistent)

Summary: {'total_records': 500, 'clean_records': 400, 'missing_resolution': np.int64(100), 'avg_resolution': np.float64(57.5), 'max_resolution': np.float64(120.0)}
Consistency check: True
